In [1]:
from skimage import measure
import numpy as np
import pandas as pd
from PIL import Image
import json
import re
import glob
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path
import shutil
import random


In [9]:
def filter_optimam_csv(csv_path, csv_out_path):
    """
    Filter the Optimam CSV file to only include relevant columns and rows.
    """
    # Read the CSV file
    df = pd.read_csv(csv_path)

    # Filter the DataFrame to only include rows where the columns x1, x2, y1, and y2 are not null
    df = df.dropna(subset=['x1', 'x2', 'y1', 'y2'])

    # Filter rows to include only rows where they contain '*mass*' in pathologies column
    df = df[df['pathologies'].str.contains('mass', case=False, na=False)]

    # Save the filtered DataFrame to a new CSV file
    df.to_csv(csv_out_path, index=False)


filtered_df = filter_optimam_csv('/home/albert/datasets/optimam/optimam_dataset.csv', 'optimam_filtered.csv')

Unique client IDs:  ['demd2841' 'demd108109' 'demd5792' ... 'demd47982' 'demd6347' 'demd7374']
Number of unique client IDs:  6000
Number of rows in the dataset:  47542
Columns in the dataset:  Index(['client_id', 'status', 'site', 'study_id', 'serie_id', 'image_id',
       'view', 'laterality', 'age', 'mark_id', 'lesion_id', 'conspicuity',
       'x1', 'x2', 'y1', 'y2', 'pathologies', 'manufacturer', 'pixel_spacing',
       'magnification_factor', 'implant', 'xmin_cropped', 'xmax_cropped',
       'ymin_cropped', 'ymax_cropped'],
      dtype='object')
Number of rows in the dataset after filtering:  4318
Unique client IDs:  ['demd2841' 'demd5792' 'demd115271' ... 'demd5935' 'demd7499' 'demd7374']
Number of unique client IDs:  2264


In [2]:
def filter_and_resize_images(csv_path, images_root, output_root):
    df = pd.read_csv(csv_path)

    os.makedirs(output_root, exist_ok=True)

    num_errors = 0
    not_found = 0
    saved_images = 0

    scale_x_list = []
    scale_y_list = []
    resized_width_list = []
    resized_height_list = []

    for _, row in df.iterrows():
        client_id = str(row['client_id'])
        study_id = str(row['study_id'])
        image_id = str(row['image_id'])

        original_image_path = os.path.join(images_root, client_id, study_id, f"{image_id}.png")

        if os.path.exists(original_image_path):
            try:
                img = Image.open(original_image_path)
                original_width, original_height = img.width, img.height

                # Redimensionamos manteniendo aspecto
                aspect_ratio = original_width / original_height
                if original_width > original_height:
                    new_width = 900
                    new_height = int(new_width / aspect_ratio)
                else:
                    new_height = 900
                    new_width = int(new_height * aspect_ratio)

                # Calculamos escalado
                scale_x = new_width / original_width
                scale_y = new_height / original_height

                img_resized = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

                # Guardamos la imagen redimensionada
                output_dir = os.path.join(output_root, client_id, study_id)
                os.makedirs(output_dir, exist_ok=True)
                output_path = os.path.join(output_dir, f"{image_id}.png")
                img_resized.save(output_path)
                saved_images += 1

                # Guardamos escalados
                scale_x_list.append(scale_x)
                scale_y_list.append(scale_y)
                resized_width_list.append(new_width)
                resized_height_list.append(new_height)

            except Exception as e:
                print(f"Error procesando {original_image_path}: {e}")
                num_errors += 1
                scale_x_list.append(None)
                scale_y_list.append(None)
                resized_width_list.append(None)
                resized_height_list.append(None)
        else:
            print(f"Imagen no encontrada: {original_image_path}")
            not_found += 1
            scale_x_list.append(None)
            scale_y_list.append(None)
            resized_width_list.append(None)
            resized_height_list.append(None)

    print(f"Number of errors: {num_errors}")
    print(f"Number of images not found: {not_found}")
    print(f"Number of images saved: {saved_images}")

    # Añadimos columnas nuevas al dataframe para los escalados de los bbox
    df['scale_x'] = scale_x_list
    df['scale_y'] = scale_y_list
    df['resized_width'] = resized_width_list
    df['resized_height'] = resized_height_list

    new_csv_path = os.path.splitext(csv_path)[0] + '_bbox_resized.csv'
    df.to_csv(new_csv_path, index=False)
    print(f"Nuevo CSV guardado en: {new_csv_path}")

csv_path = '/home/albert/research/retinanet/pytorch-retinanet_old_version/optimam_filtered.csv'
images_root = '/home/kaisar/Datasets/OPTIMAM/images'
output_root = '/home/albert/datasets/optimam/images'

filter_and_resize_images(csv_path, images_root, output_root)


Number of errors: 0
Number of images not found: 0
Number of images saved: 4318
Nuevo CSV guardado en: /home/albert/research/retinanet/pytorch-retinanet_old_version/optimam_filtered_bbox_resized.csv


In [4]:
def copiar_imagenes_a_directorio_plano(csv_path, images_root, output_dir):
    df = pd.read_csv(csv_path)

    os.makedirs(output_dir, exist_ok=True)

    for _, row in df.iterrows():
        client_id = str(row["client_id"])
        study_id = str(row["study_id"])
        image_id = str(row["image_id"])

        # Ruta original esperada
        image_filename = f"{image_id}.png"
        original_path = os.path.join(images_root, client_id, study_id, image_filename)

        # Nombre nuevo de imagen
        new_filename = f"{client_id}_{study_id}_{image_id}.png"
        output_path = os.path.join(output_dir, new_filename)

        if os.path.exists(original_path):
            shutil.copy2(original_path, output_path)
        else:
            print(f"Imagen no encontrada: {original_path}")

    print(f"Imágenes copiadas a {output_dir}")

# Ejemplo de uso:
copiar_imagenes_a_directorio_plano(
    csv_path = '/home/albert/research/retinanet/pytorch-retinanet_old_version/optimam_filtered_bbox_resized.csv',
    images_root="/home/albert/datasets/optimam/images",
    output_dir="/home/albert/datasets/optimam/images_flat"
)

✅ Imágenes copiadas a /home/albert/datasets/optimam/images_flat


In [6]:
def generate_coco_json(csv_path, images_dir, output_json_path):
    df = pd.read_csv(csv_path)

    images = []
    annotations = []
    annotation_id = 0
    image_id_map = {}
    image_id_counter = 0

    for _, row in df.iterrows():
        client_id = str(row['client_id'])
        study_id = str(row['study_id'])
        image_id_str = str(row['image_id'])
        dataset_id = client_id

        file_name = f"{client_id}_{study_id}_{image_id_str}.png"
        file_path = os.path.join(images_dir, file_name)

        if not os.path.exists(file_path):
            print(f"Imagen no encontrada: {file_path}")
            continue

        # Resized dimensions
        width = int(row['resized_width'])
        height = int(row['resized_height'])

        scale_x = row['scale_x']
        scale_y = row['scale_y']

        # Bounding box original
        xmin = float(row['x1'])
        xmax = float(row['x2'])
        ymin = float(row['y1'])
        ymax = float(row['y2'])

        # Bounding box escalado
        bbox_x = xmin * scale_x
        bbox_y = ymin * scale_y
        bbox_w = (xmax - xmin) * scale_x
        bbox_h = (ymax - ymin) * scale_y
        area = bbox_w * bbox_h

        # Añadimos imagen si aún no está registrada
        unique_key = (client_id, study_id, image_id_str)
        if unique_key not in image_id_map:
            image_info = {
                "id": image_id_counter,
                "Dataset_ID": dataset_id,
                "file_name": file_name,
                "width": width,
                "height": height
            }
            images.append(image_info)
            image_id_map[unique_key] = image_id_counter
            image_id_counter += 1

        # Mapear pathologies a category_id 1
        category_id = 1

        annotation = {
            "id": annotation_id,
            "image_id": image_id_map[unique_key],
            "category_id": category_id,
            "bbox": [bbox_x, bbox_y, bbox_w, bbox_h],
            "type": "mass",
            "area": area,
            "iscrowd": 0
        }
        annotations.append(annotation)
        annotation_id += 1

    categories = [
        {
            "id": 1,
            "name": "ROI",
            "supercategory": "mass_MALIGNANT"
        },
        {
            "id": 2,
            "name": "ROI",
            "supercategory": "mass_BENIGN"
        },
        {
            "id": 3,
            "name": "ROI",
            "supercategory": "mass_BENIGN_WITHOUT_CALLBACK"
        }
    ]

    coco_format = {
        "images": images,
        "annotations": annotations,
        "categories": categories
    }

    with open(output_json_path, 'w') as f:
        json.dump(coco_format, f, indent=4)

    print(f"COCO JSON generado con {len(images)} imágenes y {len(annotations)} anotaciones.")

generate_coco_json(
    csv_path='/home/albert/research/retinanet/pytorch-retinanet_old_version/optimam_filtered_bbox_resized.csv',
    images_dir='/home/albert/datasets/optimam/images_flat',
    output_json_path='/home/albert/research/retinanet/pytorch-retinanet_old_version/coco/annotations/coco_optimam.json'
)


COCO JSON generado con 4136 imágenes y 4318 anotaciones.


In [7]:
def dividir_y_guardar_imagenes(images_flat_dir, dest_root):
    # Obtener todas las imágenes
    imagenes = [f for f in os.listdir(images_flat_dir) if f.endswith('.png')]
    random.shuffle(imagenes)

    total = len(imagenes)
    test_size = int(0.20 * total)
    val_size = int(0.20 * (total - test_size))  # 20% del 80% restante

    test_imgs = imagenes[:test_size]
    val_imgs = imagenes[test_size:test_size + val_size]
    train_imgs = imagenes[test_size + val_size:]

    # Rutas destino
    dirs = {
        "train": os.path.join(dest_root, "optimam_coco_mass_train"),
        "val": os.path.join(dest_root, "optimam_coco_mass_val"),
        "test": os.path.join(dest_root, "optimam_coco_mass_test")
    }

    # Crear carpetas si no existen
    for path in dirs.values():
        os.makedirs(path, exist_ok=True)

    # Función para copiar
    def copiar(lista, destino):
        for img in lista:
            src = os.path.join(images_flat_dir, img)
            dst = os.path.join(destino, img)
            shutil.copy2(src, dst)

    # Copiar imágenes
    copiar(train_imgs, dirs["train"])
    copiar(val_imgs, dirs["val"])
    copiar(test_imgs, dirs["test"])

    print(f"✅ {len(train_imgs)} imágenes en train")
    print(f"✅ {len(val_imgs)} imágenes en val")
    print(f"✅ {len(test_imgs)} imágenes en test")

# Uso
dividir_y_guardar_imagenes(
    images_flat_dir="/home/albert/datasets/optimam/images_flat",
    dest_root="/home/albert/research/retinanet/pytorch-retinanet_old_version/coco/images"
)

✅ 2648 imágenes en train
✅ 661 imágenes en val
✅ 827 imágenes en test


In [8]:
def generar_coco_split(coco_global_path, images_split_dir, output_json_path):
    with open(coco_global_path, 'r') as f:
        coco_data = json.load(f)

    split_filenames = set(os.listdir(images_split_dir))

    # Filtramos las imágenes del COCO que están en este split
    imagenes_split = [
        img for img in coco_data["images"]
        if img["file_name"] in split_filenames
    ]

    # Obtenemos los IDs de imagenes seleccionadas
    split_image_ids = set(img["id"] for img in imagenes_split)

    # Filtramos las anotaciones correspondientes
    anotaciones_split = [
        ann for ann in coco_data["annotations"]
        if ann["image_id"] in split_image_ids
    ]

    # Mantenemos las categorías igual
    categorias = coco_data["categories"]

    # Contruimos el json final
    coco_split = {
        "images": imagenes_split,
        "annotations": anotaciones_split,
        "categories": categorias
    }

    with open(output_json_path, 'w') as f:
        json.dump(coco_split, f, indent=4)

    print(f"Guardado COCO para split en: {output_json_path}")
    print(f"Imágenes: {len(imagenes_split)} | Anotaciones: {len(anotaciones_split)}")


generar_coco_split(
    coco_global_path="/home/albert/research/retinanet/pytorch-retinanet_old_version/coco/annotations/coco_optimam.json",
    images_split_dir="/home/albert/research/retinanet/pytorch-retinanet_old_version/coco/images/optimam_coco_mass_train",
    output_json_path="/home/albert/research/retinanet/pytorch-retinanet_old_version/coco/annotations/instances_optimam_coco_mass_train.json"
)

generar_coco_split(
    coco_global_path="/home/albert/research/retinanet/pytorch-retinanet_old_version/coco/annotations/coco_optimam.json",
    images_split_dir="/home/albert/research/retinanet/pytorch-retinanet_old_version/coco/images/optimam_coco_mass_val",
    output_json_path="/home/albert/research/retinanet/pytorch-retinanet_old_version/coco/annotations/instances_optimam_coco_mass_val.json"
)

generar_coco_split(
    coco_global_path="/home/albert/research/retinanet/pytorch-retinanet_old_version/coco/annotations/coco_optimam.json",
    images_split_dir="/home/albert/research/retinanet/pytorch-retinanet_old_version/coco/images/optimam_coco_mass_test",
    output_json_path="/home/albert/research/retinanet/pytorch-retinanet_old_version/coco/annotations/instances_optimam_coco_mass_test.json"
)


✅ Guardado COCO para split en: /home/albert/research/retinanet/pytorch-retinanet_old_version/coco/annotations/instances_optimam_coco_mass_train.json
   Imágenes: 2648 | Anotaciones: 2762
✅ Guardado COCO para split en: /home/albert/research/retinanet/pytorch-retinanet_old_version/coco/annotations/instances_optimam_coco_mass_val.json
   Imágenes: 661 | Anotaciones: 693
✅ Guardado COCO para split en: /home/albert/research/retinanet/pytorch-retinanet_old_version/coco/annotations/instances_optimam_coco_mass_test.json
   Imágenes: 827 | Anotaciones: 863


Campo 'segmentation' eliminado correctamente. Archivo guardado en: ./coco/annotations/instances_coco_mass_val_no_segmentation.json


In [19]:
# Plot de TPR\FPPI con Min of the Max FPPI y original

os.makedirs('resultados_metricas', exist_ok=True)
csv_files = glob.glob('resultados_metricas/*.csv')
plt.figure(figsize=(8, 6))

all_max_fppi = []

for file in csv_files:
    if "to" in file:
        continue
    df = pd.read_csv(file)
    label = os.path.splitext(os.path.basename(file))[0]
    all_max_fppi.append(df['FPPI'].max())

    plt.plot(df['FPPI'], df['TPR'], label=label)


plt.xlabel('FPPI')
plt.ylabel('TPR')
plt.title('Curvas FROC experimentos sin Transfer Learning')
plt.grid(True)
plt.legend(title_fontsize='small', loc='lower right', fontsize='small')
plt.xlim(0, min(all_max_fppi))
# plt.ylim(0, 1)
plt.tight_layout()

# Guardar figura en carpeta
# plt.savefig('resultados_metricas/froc_curvas_original_all.png', dpi=300)
plt.savefig('resultados_metricas/froc_curvas_xLimited_to_min_of_max_fppi_no_TL.png', dpi=300)
plt.close()

In [22]:
# Para graficar las curvas FROC con FPPI normalizado y AUC desde el primer FPPI mayor que 0
from sklearn.metrics import auc

os.makedirs('resultados_metricas', exist_ok=True)
csv_files = glob.glob('resultados_metricas/*.csv')
plt.figure(figsize=(8, 6))

all_max_fppi = []
dfs = []

for file in csv_files:
    if "to" not in file:
        continue
    df = pd.read_csv(file)
    all_max_fppi.append(df['FPPI'].max())
    dfs.append((os.path.splitext(os.path.basename(file))[0], df))

min_max_fppi = min(all_max_fppi)

for label, df in dfs:
    df_filtered = df[df['FPPI'] <= min_max_fppi].copy()
    df_filtered['FPPI_norm'] = df_filtered['FPPI'] / min_max_fppi

    # Ordenamos por FPPI normalizado
    df_filtered = df_filtered.sort_values('FPPI_norm')

    # Encontramos el primer FPPI > 0
    df_nonzero = df_filtered[df_filtered['FPPI_norm'] > 0]
    if not df_nonzero.empty:
        first_fppi = df_nonzero['FPPI_norm'].iloc[0]
        first_tpr = df_nonzero['TPR'].iloc[0]

        auc_x = [first_fppi] + list(df_nonzero['FPPI_norm'])
        auc_y = [0] + list(df_nonzero['TPR'])

        auc_score = auc(auc_x, auc_y)
    else:
        auc_score = 0.0  # No hay FPPI > 0

    plt.plot(df_filtered['FPPI_norm'], df_filtered['TPR'], label=f"{label} (AUC={auc_score:.3f})")

plt.xlabel('FPPI')
plt.ylabel('TPR')
plt.title('Curvas FROC experimentos con Transfer Learning')
plt.grid(True)
plt.legend(title_fontsize='small', loc='lower right', fontsize='small')
plt.xlim(0, 1)
plt.tight_layout()

plt.savefig('resultados_metricas/froc_curvas_fppi_normalizado_auc_from_first_positive_TL.png', dpi=300)
# plt.savefig('resultados_metricas/froc_curvas_fppi_normalizado_auc_from_first_positive_no_TL.png', dpi=300)
plt.close()


In [23]:
# Para imprimir TPR interpolado en FPPI específicos desde los CSVs de resultados

os.makedirs('resultados_metricas', exist_ok=True)
csv_files = glob.glob('resultados_metricas/*.csv')

# Puntos de FPPI que queremos analizar
target_fppi = [0.0, 0.5, 0.7, 1.0]

print("TPR interpolado en FPPI específicos:\n")

for file in csv_files:
    df = pd.read_csv(file)
    label = os.path.splitext(os.path.basename(file))[0]

    # Asegurarse de que los datos estén ordenados por FPPI
    df_sorted = df.sort_values(by='FPPI')

    # Interpolación
    interpolated_tpr = np.interp(target_fppi, df_sorted['FPPI'], df_sorted['TPR'])

    print(f"Modelo: {label}")
    for fppi_val, tpr_val in zip(target_fppi, interpolated_tpr):
        print(f"  FPPI={fppi_val:.1f} → TPR={tpr_val:.3f}")
    print()

TPR interpolado en FPPI específicos:

Modelo: CBIS-DDSM_TL_to_OPTIMAM
  FPPI=0.0 → TPR=0.000
  FPPI=0.5 → TPR=0.794
  FPPI=0.7 → TPR=0.810
  FPPI=1.0 → TPR=0.810

Modelo: OPTIMAM_to_CBIS-DDSM
  FPPI=0.0 → TPR=0.000
  FPPI=0.5 → TPR=0.602
  FPPI=0.7 → TPR=0.652
  FPPI=1.0 → TPR=0.677

Modelo: CBIS-DDSM_to_OPTIMAM
  FPPI=0.0 → TPR=0.000
  FPPI=0.5 → TPR=0.661
  FPPI=0.7 → TPR=0.674
  FPPI=1.0 → TPR=0.674

Modelo: CBIS-DDSM_SI_DA_SI_Pretrained
  FPPI=0.0 → TPR=0.000
  FPPI=0.5 → TPR=0.638
  FPPI=0.7 → TPR=0.669
  FPPI=1.0 → TPR=0.682

Modelo: CBIS-DDSM_NO_DA_NO_Pretrained
  FPPI=0.0 → TPR=0.006
  FPPI=0.5 → TPR=0.365
  FPPI=0.7 → TPR=0.429
  FPPI=1.0 → TPR=0.465

Modelo: OPTIMAM
  FPPI=0.0 → TPR=0.000
  FPPI=0.5 → TPR=0.817
  FPPI=0.7 → TPR=0.817
  FPPI=1.0 → TPR=0.817

Modelo: CBIS-DDSM_NO_DA_SI_Pretrained
  FPPI=0.0 → TPR=0.000
  FPPI=0.5 → TPR=0.632
  FPPI=0.7 → TPR=0.674
  FPPI=1.0 → TPR=0.688



In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def graficar_bbox_dimensions(csv_path, test_images_dir, output_path):
    df = pd.read_csv(csv_path)

    df['file_name'] = df.apply(lambda row: f"{row['client_id']}_{row['study_id']}_{row['image_id']}.png", axis=1)

    # Filtramos por imágenes en el set de test
    test_filenames = set(os.listdir(test_images_dir))
    df_test = df[df['file_name'].isin(test_filenames)]

    # Calculamos bbox width y height escalados
    bbox_w = (df_test['x2'] - df_test['x1']) * df_test['scale_x']
    bbox_h = (df_test['y2'] - df_test['y1']) * df_test['scale_y']
    
    plt.figure(figsize=(8, 6))
    plt.scatter(bbox_w, bbox_h, alpha=0.6, edgecolors='k')
    plt.xlabel('Bounding Box Width (px)')
    plt.ylabel('Bounding Box Height (px)')
    plt.title('Distribución de tamaños de Bounding Boxes (Test Set)')
    plt.grid(True)

    # Crear carpeta de salida si no existe
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Guardar figura
    plt.savefig(output_path, dpi=300)
    plt.close()

# Uso
graficar_bbox_dimensions(
    csv_path='/home/albert/research/retinanet/pytorch-retinanet_old_version/optimam_filtered_bbox_resized.csv',
    test_images_dir='/home/albert/research/retinanet/pytorch-retinanet_old_version/coco/images/optimam_coco_mass_test',
    output_path='/home/albert/research/retinanet/pytorch-retinanet_old_version/resultados_metricas/test_image_size_distribution.png'
)


Escalado de bounding boxes:
Escalado X: [0.16805697 0.21965144 0.2703125  0.29365079 0.2202524  0.39184953
 0.29296875 0.25497866 0.25571429 0.32132565]
863 bounding boxes en el set de test
0     222.339369
9      75.340445
12     53.610173
14     33.518750
18     73.795313
20     71.632812
21     37.843750
22     47.575000
23     37.573438
24    103.800000
dtype: float64

0     254.621849
9      66.796875
12     48.235294
14     29.206731
18     74.639423
20     57.331731
21     46.243990
22     38.671875
23     25.420673
24    109.254808
dtype: float64


In [1]:
# Para graficar la distribución de tamaños de BBoxes correctos vs incorrectos

def graficar_bbox_correct_vs_wrong(correct_csv_path, wrong_csv_path, output_path):
    df_correct = pd.read_csv(correct_csv_path)
    df_wrong = pd.read_csv(wrong_csv_path)

    # Calculamos las dimensiones escaladas
    w_correct = df_correct['width'] * df_correct['scale']
    h_correct = df_correct['height'] * df_correct['scale']

    w_wrong = df_wrong['width'] * df_wrong['scale']
    h_wrong = df_wrong['height'] * df_wrong['scale']

    plt.figure(figsize=(8, 6))
    plt.scatter(w_correct, h_correct, color='blue', alpha=0.6, edgecolors='k',
                label=f'TP ({len(df_correct)})')
    plt.scatter(w_wrong, h_wrong, color='red', alpha=0.6, edgecolors='k',
                label=f'FN ({len(df_wrong)})')

    plt.xlabel('Bounding Box Width (px)')
    plt.ylabel('Bounding Box Height (px)')
    plt.title('Distribución de tamaños de BBoxes - TP vs FN')
    plt.legend()
    plt.grid(True)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    plt.savefig(output_path, dpi=300)
    plt.close()

graficar_bbox_correct_vs_wrong(
    correct_csv_path='/home/albert/research/retinanet/pytorch-retinanet_old_version/gt_pred_correct.csv',
    wrong_csv_path='/home/albert/research/retinanet/pytorch-retinanet_old_version/gt_pred_incorrect.csv',
    output_path='/home/albert/research/retinanet/pytorch-retinanet_old_version/resultados_metricas/bbox_size_distribution_comparison.png'
)
